In [1]:
import csv
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
import time
import random
import json

In [2]:
headers = {
        'Authority':'www.mafengwo.cn',
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate,br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Connection': 'keep-alive',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
        }

In [3]:
df = pd.read_csv('mafengwo_url.csv')

In [4]:
name_list = list(df['Name'])
url_list = list(df['Url'])            #把获取回来的panda的数据，转换成list列表
ary_member = []                       #ary_member用于保存数据字典的数组
for i in range(0,len(url_list)):      #循环把url_list赋值到dic_member字典内,并加入ary_member数组中
    dic_member = {}
    dic_member['Name'] = name_list[i]
    dic_member['Url'] = url_list[i]
    ary_member.append(dic_member)

In [5]:
len(ary_member)           #输出总成员人数

514

In [6]:
def Crawl_members_of_the_Comment(member):
    url = member['Url'][:-5] + '/review.html'
    try:
        res = requests.get(url, headers=headers)       #生成请求,并加上头,获取请求信息
        soup_bs = BeautifulSoup(res.text, 'lxml')      #通过BeautifulSoup获取html
    except:
        print('Error:The user information url request failed!' + '(' + member['Name'] + ')')    #输出错误信息

    try:
        block = soup_bs.find('li',class_='num-reviews _j_filter on') #爬取点评数
        comment = block.find('a',role='button')
        num_comment = int(comment.find('b').text)
        member['Num'] = num_comment  
    except:
        member['Num'] = None
        print('Error:The comment number request failed!' + '(' + member['Name'] + ')')
        
    try:
        block = soup_bs.find('div',class_='reviews-list')
        Crawl_data(block)
    except:
        print('Error:The block climb out failed!' + '(' + member['Name'] + ')')
    
    if(member['Num'] != None):
        Crawl_members_of_the_Comment_json(member)

In [7]:
def Crawl_members_of_the_Comment_json(member):
    member_id = member['Url'][25:-5]
    member_num = int(member['Num'] / 20)
    for i in range(1,member_num+1):
        num = i * 20
        re_url = 'http://www.mafengwo.cn/home/ajax_review.php?act=loadList&filter=0&offset={member_num}&limit=20&uid={member_id}&sort=1'.format(member_num=num, member_id=member_id)
        time.sleep(random.randint(1,3))
        try:
            res = requests.get(re_url, headers=headers)    #生成请求,并加上头,获取请求信息
            json_html = json.loads(res.text)
            html = json_html['data']['html']
            soup_bs = BeautifulSoup(html,'lxml')
            Crawl_data(soup_bs)
        except:
            print('Error:The json request failed!' + '(' + member['Name'] + ')')

In [8]:
def Crawl_data(soup_bs):
    for item in soup_bs.find_all('div',class_='poi-detail'):
        dic_comment = {}
        try:
            str_title = item.find('h3',class_='title')
            if('<br/>' in str(str_title)):
                str_title_en = str_title.find('span').text
                len_title_en = len(str_title_en)
                str_title_cn = str_title.text[:-len_title_en]
            else:
                str_title_cn = str_title.text
                str_title_en = None
            dic_comment['Title_cn'] = str_title_cn
            dic_comment['Title_en'] = str_title_en
        except:
            dic_comment['Title_cn'] = None
            dic_comment['Title_en'] = None
            print('Error:The title climb out failed!')

        try:
            str_star = item.find('div',class_='rating').get('data-star')
            dic_comment['Star'] = str_star + '星'
        except:
            dic_comment['Star'] = None
            print('Error:The star climb out failed!')
  
        ary_comment.append(dic_comment)

In [9]:
ary_comment = []
for member in ary_member:
    Crawl_members_of_the_Comment(member)

Error:The comment number request failed!(马蜂窝用户())
Error:The block climb out failed!(马蜂窝用户())
Error:The comment number request failed!(jecica0863(魔都))
Error:The block climb out failed!(jecica0863(魔都))
Error:The comment number request failed!(马蜂窝用户())
Error:The block climb out failed!(马蜂窝用户())


In [10]:
len(ary_comment)

24183

In [11]:
df_new = pd.DataFrame(ary_comment,columns=['Star','Title_cn','Title_en'])
df_new.to_csv('mafengwo_comment.csv',index=False,encoding='utf_8_sig')

In [12]:
df_new

,Star,Title_cn,Title_en
0,5星,Settlement on Quay,Settlement on Quay
1,5星,毕德寮(富力盈通店),None
2,5星,宾宾酒家(34年老店),None
3,5星,无牌咖啡,None
4,5星,风味馆,None
5,5星,荣生集北水蛇粥(光孝路店),None
6,5星,南信牛奶甜品专家,None
7,5星,陈添记(十五甫三巷店),None
8,4星,南园酒家,None
9,5星,老碗(湘子庙街店),None
